In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

In [2]:
llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [3]:
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history"
)

In [4]:
examples = [
    {
        "movie": "Top Gun",
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "movie": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝"
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Convert {movie} into three emojis that represent the movie."),
        ("ai", "{answer}"),
    ]
)

fewshot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You reply only three emojis from taking the title of a movie."),
        fewshot_chat_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [8]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

chain.invoke({"input": "Slamdunk"})

🏀🔥🏆

AIMessageChunk(content='🏀🔥🏆')

In [9]:
chain.invoke({"input": "What movie did I ask about?"})

🍝🔫👨‍👨‍👦

AIMessageChunk(content='🍝🔫👨\u200d👨\u200d👦')

In [10]:
def invoke_chain(input):
    result = chain.invoke({"input": input})
    memory.save_context(
        {"input": input}, 
        {"output": result.content}
    )
    print(result)

invoke_chain("The lord of the rings")

🧙‍♂️💍🗡️content='🧙\u200d♂️💍🗡️'


In [11]:
invoke_chain("Parasite")

🏠👨‍👩‍👧‍👦🪳content='🏠👨\u200d👩\u200d👧\u200d👦🪳'


In [12]:
invoke_chain("What movies did I ask about?")

"Parasite" 🏠👨‍👩‍👧‍👦🪳content='"Parasite" 🏠👨\u200d👩\u200d👧\u200d👦🪳'
